In [1]:
import pandas as pd
import numpy as np
import datetime


from sklearn.ensemble import RandomForestClassifier

In [2]:
veg_rem_combined_df = pd.read_csv(r'../data/FW_Veg_Rem_Combined.csv')

In [5]:
veg_rem_combined_df.columns.values, 
veg_rem_combined_df.head(2)

,Unnamed: 0,Unnamed: 0.1,fire_name,fire_size,fire_size_class,stat_cause_descr,latitude,longitude,state,disc_clean_date,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
0,0,0,NaN,10.0,C,Missing/Undefined,18.105072,-66.753044,PR,2/11/2007,...,3.250413,78.21659,76.793750,76.381579,78.724370,0.0,0.0,0.0,0.0,0.017923
1,1,1,NaN,3.0,B,Arson,35.038330,-87.610000,TN,12/11/2006,...,2.122320,70.84000,65.858911,55.505882,81.682678,59.8,8.4,0.0,86.8,0.184355


In [14]:
#veg_rem_combined_df.iloc[0]

In [6]:
fire_df = veg_rem_combined_df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', ])

In [7]:
fire_df.head()

,fire_name,fire_size,fire_size_class,stat_cause_descr,latitude,longitude,state,disc_clean_date,cont_clean_date,discovery_month,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
0,NaN,10.0,C,Missing/Undefined,18.105072,-66.753044,PR,2/11/2007,NaN,Feb,...,3.250413,78.216590,76.793750,76.381579,78.724370,0.0,0.0,0.0,0.0,0.017923
1,NaN,3.0,B,Arson,35.038330,-87.610000,TN,12/11/2006,NaN,Dec,...,2.122320,70.840000,65.858911,55.505882,81.682678,59.8,8.4,0.0,86.8,0.184355
2,NaN,60.0,C,Arson,34.947800,-88.722500,MS,2/29/2004,NaN,Feb,...,3.369050,75.531629,75.868613,76.812834,65.063800,168.8,42.2,18.1,124.5,0.194544
3,WNA 1,1.0,B,Debris Burning,39.641400,-119.308300,NV,6/6/2005,6/6/2005,Jun,...,0.000000,44.778429,37.140811,35.353846,0.000000,10.4,7.2,0.0,0.0,0.487447
4,NaN,2.0,B,Miscellaneous,30.700600,-90.591400,LA,9/22/1999,NaN,Sep,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.214633


In [54]:
humidity_df = pd.read_csv(r'../data/humidity.csv')

In [55]:
humidity_df.head(2)

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN
1,2012-10-01 13:00:00,76.0,81.0,88.0,81.0,88.0,82.0,22.0,23.0,50.0,...,71.0,58.0,93.0,68.0,50.0,63.0,22.0,51.0,51.0,50.0


In [56]:
humidity_df['datetime'] = pd.to_datetime(humidity_df['datetime']).apply(datetime.datetime.date)
humidity_df = humidity_df.groupby('datetime').mean()

In [57]:
humidity_df

,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
datetime,,,,,,,,,,,,,,,,,,,,,
2012-10-01,78.272727,78.727273,83.000000,78.000000,88.000000,79.909091,20.727273,24.181818,48.909091,63.181818,...,68.272727,56.272727,75.727273,68.000000,54.818182,62.909091,23.250000,51.000,51.00,50.000000
2012-10-02,77.375000,65.833333,65.958333,65.166667,64.916667,65.875000,16.166667,24.476190,40.363636,66.647059,...,54.500000,50.950000,39.500000,68.750000,72.916667,69.708333,33.833333,53.500,53.50,52.750000
2012-10-03,71.625000,66.208333,47.083333,58.416667,44.875000,59.750000,14.500000,18.380952,31.904762,36.869565,...,78.791667,68.541667,88.400000,86.041667,75.541667,77.208333,47.291667,69.375,69.75,62.291667
2012-10-04,49.000000,51.166667,60.166667,48.869565,66.625000,75.583333,15.166667,20.238095,19.681818,14.750000,...,88.565217,94.545455,94.750000,84.750000,67.458333,64.625000,49.333333,71.250,71.25,63.375000
2012-10-05,43.409091,40.391304,69.666667,48.521739,70.250000,72.791667,14.647059,27.777778,18.250000,49.500000,...,84.750000,93.041667,88.000000,73.333333,65.666667,63.416667,46.875000,70.500,70.50,67.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-11-26,NaN,87.625000,NaN,93.291667,86.666667,88.250000,38.166667,43.208333,29.750000,47.833333,...,53.125000,NaN,78.166667,54.666667,NaN,NaN,NaN,NaN,NaN,NaN
2017-11-27,NaN,92.875000,NaN,85.416667,77.333333,81.333333,34.458333,34.041667,29.041667,25.958333,...,49.375000,NaN,74.125000,55.958333,NaN,NaN,NaN,NaN,NaN,NaN
2017-11-28,NaN,88.458333,NaN,85.750000,54.958333,76.375000,33.125000,46.208333,31.458333,47.666667,...,54.666667,NaN,64.833333,57.541667,NaN,NaN,NaN,NaN,NaN,NaN
